# <center> RNA-Seq Differential Gene Expression Notebook </center>

## <center>Preface</center>  
<p><center>This Notebook provides an interactive way to process, analyze, and view differential gene expressions from next-generation sequencing data.<br>The following steps serve as a guide, however, care should be taken when executing and interpreting results.</center><br>This notebook uses the following R packages: <li> - <a href="https://bioconductor.org/packages/release/bioc/html/Rsubread.html"><strong>Rsubread</strong></a> to produce a count matrix from provided .bam files using featureCounts (if matrix not already created). <br>- <a href="https://bioconductor.org/packages/release/bioc/html/DESeq2.html"><strong>DESeq2</strong></a> to process the count matrix and produce differential gene expression statistics. <br>- <a href="https://bioconductor.org/packages/release/bioc/html/clusterProfiler.html"><strong>clusterProfiler</strong></a> to annotate genes (GO, GSEA, KEGG) using database packages for mappings. </li><center><br><em>Note that despite following the framework below, it is up to <strong>you</strong> to determine which steps or values to change or ignore based on your knowledge of experimental details in obtaining data and your analysis goals.</em></center></p>

### sessionInfo()

R version 4.2.2 Patched (2022-11-10 r83330)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.1 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.10.0
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C               LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8    LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C             LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    grid      stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] optparse_1.7.3                            org.Mm.eg.db_3.16.0                      
 [3] org.Hs.eg.db_3.16.0                       TxDb.Hsapiens.UCSC.hg38.knownGene_3.16.0 
 [5] TxDb.Hsapiens.UCSC.hg19.knownGene_3.2.2   TxDb.Mmusculus.UCSC.mm10.knownGene_3.10.0
 [7] GenomicFeatures_1.50.3                    AnnotationDbi_1.60.0                     
 [9] ggplot2_3.4.0                             clusterProfiler_4.6.0                    
[11] DESeq2_1.38.3                             SummarizedExperiment_1.28.0              
[13] Biobase_2.58.0                            MatrixGenerics_1.10.0                    
[15] matrixStats_0.63.0                        GenomicRanges_1.50.2                     
[17] GenomeInfoDb_1.34.6                       IRanges_2.32.0                           
[19] S4Vectors_0.36.1                          Rsubread_2.12.2                          
[21] Rgraphviz_2.42.0                          graph_1.76.0                             
[23] BiocGenerics_0.44.0                      

loaded via a namespace (and not attached):
  [1] shadowtext_0.1.2         fastmatch_1.1-3          BiocFileCache_2.6.0      plyr_1.8.8              
  [5] igraph_1.3.5             lazyeval_0.2.2           splines_4.2.2            BiocParallel_1.32.5     
  [9] digest_0.6.31            yulab.utils_0.0.6        GOSemSim_2.24.0          viridis_0.6.2           
 [13] GO.db_3.16.0             fansi_1.0.4              magrittr_2.0.3           memoise_2.0.1           
 [17] Biostrings_2.66.0        annotate_1.76.0          graphlayouts_0.8.4       enrichplot_1.18.3       
 [21] prettyunits_1.1.1        colorspace_2.1-0         blob_1.2.3               rappdirs_0.3.3          
 [25] ggrepel_0.9.2            dplyr_1.0.10             crayon_1.5.2             RCurl_1.98-1.9          
 [29] jsonlite_1.8.4           scatterpie_0.1.8         ape_5.6-2                glue_1.6.2              
 [33] polyclip_1.10-4          gtable_0.3.1             zlibbioc_1.44.0          XVector_0.38.0          
 [37] DelayedArray_0.24.0      scales_1.2.1             DOSE_3.24.2              DBI_1.1.3               
 [41] Rcpp_1.0.10              viridisLite_0.4.1        xtable_1.8-4             progress_1.2.2          
 [45] gridGraphics_0.5-1       tidytree_0.4.2           bit_4.0.5                getopt_1.20.3           
 [49] httr_1.4.4               fgsea_1.24.0             RColorBrewer_1.1-3       ellipsis_0.3.2          
 [53] pkgconfig_2.0.3          XML_3.99-0.13            farver_2.1.1             dbplyr_2.3.0            
 [57] locfit_1.5-9.7           utf8_1.2.2               ggplotify_0.1.0          tidyselect_1.2.0        
 [61] rlang_1.0.6              reshape2_1.4.4           munsell_0.5.0            tools_4.2.2             
 [65] cachem_1.0.6             downloader_0.4           cli_3.6.0                generics_0.1.3          
 [69] RSQLite_2.2.20           gson_0.0.9               stringr_1.5.0            fastmap_1.1.0           
 [73] yaml_2.3.7               ggtree_3.6.2             bit64_4.0.5              tidygraph_1.2.2         
 [77] purrr_1.0.1              KEGGREST_1.38.0          ggraph_2.1.0             nlme_3.1-161            
 [81] aplot_0.1.9              xml2_1.3.3               biomaRt_2.54.0           compiler_4.2.2          
 [85] rstudioapi_0.14          filelock_1.0.2           curl_5.0.0               png_0.1-8               
 [89] treeio_1.22.0            tibble_3.1.8             tweenr_2.0.2             geneplotter_1.76.0      
 [93] stringi_1.7.12           lattice_0.20-45          Matrix_1.5-1             vctrs_0.5.2             
 [97] pillar_1.8.1             lifecycle_1.0.3          data.table_1.14.6        cowplot_1.1.1           
[101] bitops_1.0-7             patchwork_1.1.2          rtracklayer_1.58.0       qvalue_2.30.0           
[105] R6_2.5.1                 BiocIO_1.8.0             gridExtra_2.3            codetools_0.2-18        
[109] MASS_7.3-58              assertthat_0.2.1         rjson_0.2.21             withr_2.5.0             
[113] GenomicAlignments_1.34.0 Rsamtools_2.14.0         GenomeInfoDbData_1.2.9   parallel_4.2.2          
[117] hms_1.1.2                ggfun_0.0.9              tidyr_1.3.0              HDO.db_0.99.1           
[121] ggforce_0.4.1            restfulr_0.0.15 

### Required:
<br><strong>1) A <u>COUNT MATRIX FILE</u> produced by featureCounts and/or htseq-count which should be comma-separated (.csv), example formatted as follows:</strong><br>
<table style=’float:left;’>
    <tr>
        <th>GeneID</th>
        <th>BamSample 1</th>
        <th>BamSample 2</th>
        <th>BamSample 3</th>
        <th>BamSample N</th>
    </tr>
    <tr>
        <td>gene1</td>
        <td>123</td>
        <td>456</td>
        <td>78</td>
        <td>89</td>
    </tr>
    <tr>
        <td>gene2</td>
        <td>321</td>
        <td>654</td>
        <td>45</td>
        <td>12</td>
    </tr>
    <tr>
        <td>gene3</td>
        <td>456</td>
        <td>741</td>
        <td>0</td>
        <td>0</td>
    </tr>
    <tr>
        <td>geneN</td>
        <td>369</td>
        <td>987</td>
        <td>100</td>
        <td>200</td>
    </tr>
</table><br>
&emsp;<strong>There should be no duplicated gene IDs in the first column.</strong><br>
&emsp;<strong>---This file may be produced using this notebook with the provided .bam files.---</strong>
<br><br>
<strong>2) A <u>SAMPLE INFO FILE</u> is also required and should be comma-separated (.csv), example formatted as follows:</strong><br>
<table style=’float:left;’>
    <tr>
        <th>Sample</th>
        <th>Condition</th>
    </tr>
    <tr>
        <td>BamSample 1</td>
        <td>control</td>
    </tr>
    <tr>
        <td>BamSample 2</td>
        <td>control</td>
    </tr>
    <tr>
        <td>BamSample 3</td>
        <td>experimental</td>
    </tr>
    <tr>
        <td>BamSample N</td>
        <td>experimental</td>
    </tr>
</table><br>
&emsp;<strong>Must contain a column with the header named "Condition" for grouping samples.</strong><br>
&emsp;<strong>There must be more than 1 sample for each condition.</strong><br>
&emsp;<strong>Row order does not matter.</strong><br>
&emsp;<strong>May contain multiple conditions. Pair-wise comparisons are performed for differential gene expressions between each pair.</strong>

## <center> ======= Load Packages ======= </center>

In [ ]:
suppressWarnings(suppressPackageStartupMessages({
  library(Rsubread)
  library(DESeq2)
  library(clusterProfiler)
  library(ggplot2)
  library(TxDb.Mmusculus.UCSC.mm10.knownGene)
  library(TxDb.Hsapiens.UCSC.hg19.knownGene)
  library(TxDb.Hsapiens.UCSC.hg38.knownGene)
  #library(ensembldb)
  #library(EnsDb.Hsapiens.v86)
  #library(EnsDb.Mmusculus.v79)
  library(org.Hs.eg.db)
  library(org.Mm.eg.db)
  library(optparse)
}))

---

### <center> Generate Count Matrix <br>(Skip if already provided)</center>

###  First, put all .bam (and .bai) files into a directory, e.g. "bams/", then run the following cells

In [ ]:
# Read in bam filenames from directory path
bam_files <- list.files(path="bams/", full.names=TRUE)[c(TRUE, FALSE)]

In [ ]:
# Count gene alignments (select appropriate assembly, mm10, hg38, etc...and check featureCount parameters) 
bamcounts <- featureCounts(bam_files, annot.inbuilt="mm10", ignoreDup=TRUE, isPairedEnd=TRUE, nthreads=4, verbose=TRUE)

In [ ]:
# Map gene NCBI Entrez IDs to Gene Symbols
rownames(bamcounts$counts) <- mapIds(org.Mm.eg.db, keys = rownames(bamcounts$counts), column = "SYMBOL", keytype = "ENTREZID")

In [ ]:
# Drop any unmapped genes
bamcounts$counts <- bamcounts$counts[!(is.na(rownames(bamcounts$counts))), ]

In [ ]:
# Write featureCounts output to file in current directory
for (n in names(bamcounts)){
  write.table(bamcounts[[n]], file=paste(getwd(), "/", n, ".csv", sep=""), sep=",", quote=F, col.names=NA)
}

### Check output files to ensure counts matrix was successfully created before proceeding.

---

## <center> ======= Define Variables ======= </center>

In [ ]:
# ======= Get command-line optional arguments (defaults) =======
option_list = list(
  make_option(c("-c", "--countsfile"), type="character", help="Count matrix file", metavar="character"),
  make_option(c("-s", "--sampleinfo"), type="character", help="File containing samples and their experimental conditions info (row names must match matrix file column names)", metavar="character"),
  make_option(c("-o", "--organism"), type="character", default="mouse", help="Organism to annotate genes (mouse or human)", metavar="character"),
  make_option(c("-r", "--result_dir"), type="character", default="DEG_Analysis/", help="Directory name for saving output results", metavar="character"),
  make_option(c("-f", "--filter"), action="store_true", type="logical", default=TRUE, help="Flag to filter read counts (removes 0 counts, then removes genes in bottom 10th percentile of counts for those in < 3 samples)", metavar="character"),
  make_option(c("--min_count"), type="integer", default=1, help="Minimum counts to include if filtering", metavar="integer"),
  make_option(c("--lfc"), type="double", default=1.5, help="Magnitude of log2foldchange to define significant up/down regulation of genes", metavar="integer")
  );
opt_parser = OptionParser(option_list=option_list);
opt = parse_args(opt_parser);

In [ ]:
# Path to count matrix file
opt$countsfile <- ""

In [ ]:
# Path to sample info file
opt$sampleinfo <- ""

In [ ]:
# View variables
for (i in which(names(opt) != "help")) {
  cat(names(opt)[i], '=', paste(opt)[i], "\n")
}

In [ ]:
# Manually re-define any values if defaults not desired, otherwise SKIP THIS CELL
opt$countsfile <- ""
opt$sampleinfo <- ""
opt$organism <- ""
opt$result_dir <- "DEG_Analysis/"
opt$filter <- TRUE # or FALSE
opt$min_count <- 1 # integer value
opt$lfc <- 1.5 # any numerical value

In [ ]:
# Define references for annotations
if (tolower(opt$organism) == "mouse"){
  annoDb <- "org.Mm.eg.db"
  keggOrg <- "mmu"
} else if (tolower(opt$organism) == "human"){
  annoDb <- "org.Hs.eg.db"
  keggOrg <- "hsa"
} else{
  stop("Invalid choice of organism")
}

## <center> =========== Load Input Files and Prep Run ============ </center>

In [ ]:
count_mtx <- as.matrix(read.csv(opt$countsfile, sep=",", row.names=1, check.names=FALSE))
# View 
head(count_mtx)

In [ ]:
sampleinfo <- read.csv(opt$sampleinfo, row.names=1)
# View
sampleinfo

In [ ]:
# Create output file directory and set as working directory
if (!file.exists(opt$result_dir)) {
  dir.create(opt$result_dir)
}
setwd(opt$result_dir)
cat("Output files will be in", paste(getwd(), "/", sep=""), "\n")

In [ ]:
# Get all pairwise combinations of conditions for comparing (column values shown below)
combs <- as.data.frame(combn(unique(sampleinfo$Condition), 2))
combs

## <center> =========== Run Analysis ============ </center>

In [ ]:
# Iterate through condition combinations to compare DEG
for (c in colnames(combs)){
  
  comparison <- paste(combs[[c]][1], combs[[c]][2], sep="_vs_")
  cat("\n==========", comparison, "==========\n")
  # Isolate matrix for relevant samples
  samples1 <- rownames(sampleinfo[sampleinfo$Condition == combs[[c]][1], ])
  samples2 <- rownames(sampleinfo[sampleinfo$Condition == combs[[c]][2], ])
  mtx <- count_mtx[, c(samples1, samples2)]
  mtx_info <- sampleinfo[c(samples1, samples2), ]
  mtx_info$Condition <- factor(mtx_info$Condition)
  
  # Check that matrix columns are same order as sample info
  if (!all(rownames(mtx_info) == colnames(mtx))) {
    break
  }
  
  # Create DESeqDataSet object from input
  dds <- DESeqDataSetFromMatrix(countData = mtx,
                                colData = mtx_info,
                                design = ~ Condition)
  
  # Create directory for output files of current comparison
  if (!file.exists(comparison)) {
    dir.create(comparison)
  }
  
  # Set output filenames prefix
  output_prefix <- paste(getwd(), "/", comparison, "/", sep="")
  
  cat("Genes and samples:\n", dim(dds), "\n")
  
  # =========== Filter Count Matrix ============
  if (opt$filter) {
    
    # Get percentile ranges of counts
    cat("Percentiles of genes count sums over all samples:\n")
    cat(quantile(rowSums(counts(dds)), probs = c(0, 0.1, 0.25, 0.5, 0.75, 0.9, 1)), "\n")
    
    # Remove 0 counts to avoid 0-bias and disregard irrelevant genes
    keep <- rowSums(counts(dds)) >= opt$min_count
    dds <- dds[keep,]
    cat("Percentiles after removing genes with sum(counts) <", opt$min_count, "\n")
    quantile(rowSums(counts(dds)), probs = c(0, 0.1, 0.25, 0.5, 0.75, 0.9, 1))
    # Get relevant percentiles
    quant <- quantile(rowSums(counts(dds)), probs = c(0, 0.1, 0.25, 0.5, 0.75, 0.9, 1))
    # Remove genes with <= 0.1 percentile counts
    cat("Removing genes with sum(counts) <=", quant[[2]])
    keep <- rowSums(counts(dds)) > quant[[2]]
    dds <- dds[keep,]
    # Keep genes only where there are at least 3 samples having at least that count
    cat("Removing genes with sum(counts) <=", quant[[2]], "in at least", round(length(colnames(dds))/4), "samples\n")
    keep <- rowSums(counts(dds) > quant[[2]]) >= round(length(colnames(dds))/4)
    dds <- dds[keep,]
    
    cat("Genes and samples after filtering raw counts:\n", dim(dds), "\n")
  }
  
  # ========== Normalize and Compute DEG Stats =============
  # Create DESeq object
  dds <- DESeq(dds)
  
  # Write filtered count matrix to file
  write.table(counts(dds), file=paste(output_prefix, "count_mtx_filtered.csv", sep=""), sep=",", quote=F, col.names=NA)
  
  # Write normalized values to file
  normalized_count_mtx <- counts(dds, normalized=TRUE)
  write.table(normalized_count_mtx, file=paste(output_prefix, "count_mtx_normalized.csv", sep=""), sep=",", quote=F, col.names=NA)
  
  # Obtain DEG results and output to file
  res <- results(dds, contrast=c("Condition", combs[[c]][1], combs[[c]][2]))
  res[['FoldChange']] <- 2^abs(res[['log2FoldChange']])*(res[['log2FoldChange']]/abs(res[['log2FoldChange']]))
  write.table(res[order(res$log2FoldChange, decreasing=TRUE), ], file=paste(output_prefix, "DESeq2_FullResult_", comparison, ".csv", sep=""), sep=",", quote=F, col.names=NA)
  
  #resLFC <- lfcShrink(dds, coef=resultsNames(dds)[-1], type="apeglm")
  
  # Filter out normalized low-expressed genes (bottom 10th percentile) by baseMean
  cat("Genes and samples before filtering normalized genes:\n", dim(res), "\n")
  quant <- quantile(res$baseMean, probs = c(0, 0.1, 0.25, 0.5, 0.75, 0.9, 1))
  cat("Removing genes with baseMean expression in bottom 10th percentile <=", quant[[2]], "\n")
  res <- res[res$baseMean >= quant[[2]], ]
  cat("Genes and samples after filtering normalized genes:\n", dim(res), "\n")
  
  # Generate plots
  png(paste(output_prefix, 'MAplot.png', sep=''))
  plotMA(res)
  abline(h=c((0-opt$lfc), opt$lfc), col="red", lwd=2)
  invisible(capture.output(dev.off()))
  
  #png(paste(output_prefix, 'resLFC_MAplot.png', sep=''))
  #plotMA(resLFC, ylim=c(-2,2))
  #invisible(capture.output(dev.off()))
  
  #png(paste(output_prefix, 'res_genecount_minpadj.png', sep=''))
  #plotCounts(dds, gene=which.min(res$padj), intgroup="Condition")
  #invisible(capture.output(dev.off()))
  
  vsd <- vst(dds, blind=FALSE)
  png(paste(output_prefix, 'PCAplot.png', sep=''))
  plotPCA(vsd, intgroup=c("Condition"))
  invisible(capture.output(dev.off()))
  
  # ========== Organize DEGs ==============
  # Remove any NAs
  #res <- res[rowSums(is.na(res)) == 0, ]
  
  # Get significantly Upregulated (log2FoldChange > 0)
  res_up <- res[res$log2FoldChange >= opt$lfc ,]
  res_up_sorted <- res_up[order(res_up$log2FoldChange, decreasing=TRUE),]
  write.table(res_up_sorted, file=paste(output_prefix, "DESeq2_Result_", combs[[c]][1], ".csv", sep=""), sep=",", quote=F, col.names=NA)
  
  # Get significantly Downregulated (log2FoldChange < 0)
  res_down <- res[res$log2FoldChange <= (0 - opt$lfc) ,]
  res_down_sorted <- res_down[order(res_down$log2FoldChange, decreasing=TRUE),]
  write.table(res_down[order(res_down$log2FoldChange, decreasing=FALSE),], file=paste(output_prefix, "DESeq2_Result_", combs[[c]][2], ".csv", sep=""), sep=",", quote=F, col.names=NA)
  
  # Get all significantly changed genes
  res_changed <- res[(res$log2FoldChange >= opt$lfc | res$log2FoldChange <= (0 - opt$lfc)),]
  res_changed_sorted <- res_changed[order(abs(res_changed$log2FoldChange), decreasing=TRUE),]
  write.table(res_changed_sorted, file=paste(output_prefix, "DESeq2_Result_DEGs", ".csv", sep=""), sep=",", quote=F, col.names=NA)
  
  genes <- list()
  genes[[combs[[c]][1]]] <- rownames(res_up_sorted)
  genes[[combs[[c]][2]]] <- rownames(res_down_sorted)
  genes[["DEGs"]] <- rownames(res_changed_sorted)
  
  genes_entrez <- list()
  for (n in names(genes)){
    if (tolower(opt$organism) == "human"){
      genes_entrez[[n]] <- mapIds(org.Hs.eg.db, keys = genes[[n]], column = "ENTREZID", keytype = "SYMBOL")
    }else if (tolower(opt$organism) == "mouse"){
      genes_entrez[[n]] <- mapIds(org.Mm.eg.db, keys = genes[[n]], column = "ENTREZID", keytype = "SYMBOL")
    }
  }
  
  # ============= GO and KEGG Annotations ==============
  for (n in names(genes)){
    
    # GO Annotation
    tryCatch(
      {
        compGO <- compareCluster(geneCluster=genes[n],
                                 OrgDb=annoDb,
                                 fun="enrichGO",
                                 keyType="SYMBOL", #ENSEMBL
                                 pvalueCutoff=0.05,
                                 pAdjustMethod="BH",
                                 readable=TRUE
        ) # Check https://www.genome.jp/kegg/catalog/org_list.html for organism hsa=human mmu=mouse
        if (!is.null(compGO)){
          plt <- dotplot(compGO, showCategory = 8, title = paste("GO -", n, sep=""))
          invisible(capture.output(ggsave(filename=paste(output_prefix, 'GO_annotation_', n, '.png', sep=''), plot=plt, dpi=320)))
          # Write annotations to csv
          write.table(as.data.frame(compGO), file=paste(output_prefix, 'GO_annotation_', n, '.tsv', sep=''), sep="\t", quote=F, row.names=F, col.names=T)
        }
      },error = function(e)
      {
        message(e)
      }
    )
    
    # KEGG Annotation
    tryCatch(
      {
        # fun is "groupGO", "enrichGO", "enrichKEGG", "enrichDO" or "enrichPathway" 
        compKEGG <- compareCluster(geneCluster=genes_entrez[n],
                                   fun="enrichKEGG",
                                   pvalueCutoff=0.05,
                                   pAdjustMethod="BH",
                                   organism=keggOrg
        ) # Check https://www.genome.jp/kegg/catalog/org_list.html for organism hsa=human mmu=mouse
        if (!is.null(compKEGG)){
          plt <- dotplot(compKEGG, showCategory = 8, title = paste("KEGG -", n, sep=""))
          invisible(capture.output(ggsave(filename=paste(output_prefix, 'KEGG_annotation_', n, '.png', sep=''), plot=plt, dpi=320)))
          
          # Write annotations to csv
          write.table(as.data.frame(compKEGG), file=paste(output_prefix, 'KEGG_annotation_', n, '.tsv', sep=''), sep="\t", quote=F, row.names=F, col.names=T)
        }
      },error = function(e)
      {
        message(e)
      }
    ) 
    
    # GSEA
    tryCatch(
      {
        original_gene_list <- res[genes[[n]],]$log2FoldChange
        names(original_gene_list) <- rownames(res[genes[[n]],])
        gene_list <- na.omit(original_gene_list)
        
        gsea <- gseGO(geneList=gene_list, 
                      ont ="ALL", 
                      keyType = "SYMBOL", 
                      nPerm = 10000, 
                      minGSSize = 3, 
                      maxGSSize = 800, 
                      pvalueCutoff = 0.05, 
                      verbose = TRUE, 
                      OrgDb = annoDb, 
                      pAdjustMethod = "BH"
        )
        if (!is.null(gsea)){
          plt <- dotplot(gsea, showCategory = 8, title = paste("GSEA -", n, sep=""))
          invisible(capture.output(ggsave(filename=paste(output_prefix, 'GSEA_annotation_', n, '.png', sep=''), plot=plt, dpi=320)))
          
          # Write annotations to csv
          write.table(as.data.frame(compKEGG), file=paste(output_prefix, 'GSEA_annotation_', n, '.tsv', sep=''), sep="\t", quote=F, row.names=F, col.names=T)
        }
      },error = function(e)
      {
        message(e)
      }
    ) 
  }
  
}

## <center> =========== END ============ </center>